# **Project: Analyze The Northwind Dataset**
+ After uploading the northwind Data in a AWS RD2-Cloud-Database with PostgreSQL the following commands were used to analyze the data
+ The data is about “Northwind Traders”, a fictional company. The database contains all sales transactions between the company and its customers as well as purchases from Northwinds suppliers.
+ creating different views, combine data, discover dependencies
+ the results are published within the BI Tool Metabase on AWS EC2 Server
+ The Dashboard can be inspeced under the following link: 
http://ec2-3-65-24-67.eu-central-1.compute.amazonaws.com/public/dashboard/84b5dcfa-d3b6-4e73-82d6-7a409fc7d68a

### 1. Get the names and the quantities in stock for each product.

In [ ]:
SELECT productname, unitsinstock 
FROM products;

### 2. Get a list of current products (Product ID and name).

In [ ]:
SELECT productid, productname 
FROM products;

### 3. Get a list of the most and least expensive products (name and unit price).

In [ ]:

SELECT productname, unitprice FROM products 
WHERE unitprice = (SELECT MAX(unitprice) FROM products) OR unitprice = (SELECT MIN(unitprice) FROM products);

### 4. Get products that cost less than $20.

In [ ]:
SELECT productname, unitprice FROM products 
WHERE products.unitprice < 20;

### 5. Get products that cost between $15 and $25.

In [ ]:
SELECT productname, unitprice FROM products 
WHERE products.unitprice BETWEEN 15 AND 25;

### 6. Get products above average price.

In [ ]:
SELECT productname, unitprice FROM products 
WHERE products.unitprice > (SELECT AVG(unitprice) FROM products);

### 7. Find the ten most expensive products.

In [ ]:
SELECT productname, unitprice FROM products 
ORDER BY products.unitprice DESC
LIMIT 10;

### 8. Get a list of discontinued products (Product ID and name).

In [ ]:
SELECT productid, productname  FROM products
WHERE products.discontinued = 1;

### 9. Count current and discontinued products.

In [ ]:
SELECT SUM(discontinued) AS number_discontinued , (COUNT(discontinued)-SUM(discontinued)) AS number_current FROM products;

### 10. Find products with less units in stock than the quantity on order.

In [ ]:
SELECT productname, unitsInStock, unitsOnOrder  FROM products
WHERE products.unitsInStock < products.unitsOnOrder;


### 11. Find the customer who had the highest order amount

In [ ]:
CREATE VIEW customer_order AS
SELECT cu.contactName, o.orderID 
FROM customers AS cu
FULL JOIN orders AS o
ON cu.customerID = o.customerID;

SELECT DISTINCT co.contactName, od.quantity
FROM customer_order AS co 
FULL JOIN order_details AS od
ON co.orderID = od.orderID
WHERE quantity = (SELECT MAX(quantity) FROM order_details);


### 12. Get orders for a given employee and the according customer

In [ ]:
CREATE VIEW employee_orders AS
SELECT employees.lastName, employees.firstName, orders.customerID, orders.orderID
FROM employees 
FULL JOIN orders 
ON employees.employeeID = orders.employeeID;

CREATE VIEW emp_ord_odet AS
SELECT employee_orders.lastName, employee_orders.firstName, employee_orders.customerID, order_details.productID, order_details.unitPrice, order_details.quantity, order_details.discount
FROM employee_orders
FULL JOIN order_details 
ON employee_orders.orderID = order_details.orderID;

CREATE VIEW emp_product AS
SELECT emp_ord_odet.lastName, emp_ord_odet.firstName, emp_ord_odet.customerID,products.productName, emp_ord_odet.unitPrice, emp_ord_odet.quantity, emp_ord_odet.discount 
FROM emp_ord_odet
FULL JOIN products
ON emp_ord_odet.productID = products.productID;

SELECT emp_product.lastName AS EmployeeLastName, emp_product.firstName AS EmployeeFirstName, customers.contactName AS CustomerName, emp_product.productName, emp_product.unitPrice, emp_product.quantity, emp_product.discount 
FROM emp_product
FULL JOIN customers
ON emp_product.customerID = customers.customerID
ORDER BY EmployeeLastName ASC

### 13. Find the hiring age of each employee

In [ ]:
SELECT firstName, lastName, DATE_PART('year', age(hireDat, birthDate))  AS HireAge FROM employees

### 14. Create views and/or named queries for some of these queries

In [ ]:
CREATE VIEW employee_hireage AS
SELECT firstName, lastName, DATE_PART('year', age(hireDate, birthDate))  AS HireAge FROM employees



### 15. Count total number of products

In [ ]:
SELECT COUNT(productID) AS NumberOfProducts FROM products

### 16. Separate currently available and discontinued products

In [ ]:
SELECT SUM(discontinued) AS number_discontinued , (COUNT(discontinued)-SUM(discontinued)) AS number_current FROM products;

### 17. Frequently ordered products

In [ ]:
SELECT productname, unitsOnOrder FROM products
ORDER BY products.unitsOnOrder DESC

### 18. Relative amount

In [ ]:
SELECT productname, (ROUND(unitsOnOrder*100.0 / (SELECT SUM(unitsOnOrder) FROM products),2)) AS percentage FROM products
ORDER BY percentage DESC

### 19. Search your country (or a country of choice)

In [ ]:
SELECT * FROM customers
WHERE customers.country = 'Germany'

In [ ]:
how many products by supplier

In [ ]:
CREATE VIEW Product_suppliers AS
SELECT products.productname , products.unitprice, products.unitsinstock, products.unitsonorder, products.discontinued, suppliers.companyname, suppliers.contactname, suppliers.country
FROM products
FULL JOIN suppliers
ON products.supplierid = suppliers.supplierid